In [1]:
%matplotlib inline

# ensure that any edits to libraries
# are reloaded automatically
%reload_ext autoreload
%autoreload 2

In [2]:
# make cuda_slic importable
import sys
sys.path.append("./../src/")

In [3]:
from cuda_slic.slic import slic as cuda_slic

In [4]:
import numpy as np
from skimage import data, color, filters, segmentation
from skimage.util import img_as_float32, img_as_float64
from skimage.segmentation import slic as sk_slic

## Cuda SLIC Failiure Modes
**NOTE:** all of these bugs are fixed now but I am keeping this notebook for documentation purposes.


From playing around with the parameters I was able to discover a few inputs that causes the cuda_slic algorithm to throw an error.

In [5]:
RUN_TESTS = True

### 1. input array (500, 500, 500), nps>50_000, compactness=30:

This usually throughs a `LogicError` exception. When it fails, it leaves memory residue in the GPU that is not cleaned up unless you terminate the pyhton process. 

This error is sometimes generated from the cuda compilation step of the `ccl.py` and sometimes form `slic.py`

Further complicating things the algorithm does not fail reliably with these inputs!!
However, I was able to make it fail reliably with `nps=500_000`. This indicates that the failiure rate is related to the the `nps` parameter.


In [6]:
if RUN_TESTS:
    blob = data.binary_blobs(length=600, n_dim=3, seed=2)
    blob = np.float32(blob)
    cuda_labels = cuda_slic(blob, n_segments=5_000_000, compactness=0.5)
    print(len(np.unique(cuda_labels)))

360000



```
---------------------------------------------------------------------------
LogicError                                Traceback (most recent call last)
<ipython-input-39-ad381aadb072> in <module>
      1 blob = data.binary_blobs(length=500, n_dim=3, seed=2)
      2 blob = img_as_float32(blob)
----> 3 cuda_labels = cuda_slic(blob, nsp=50_000, compactness=30)
      4 print(len(np.unique(cuda_labels)))

~/Projects/gpu-slic/survos2/improc/utils.py in wrapper(out, src_mode, *args, **kwargs)
    563                          fillvalue=fillvalue, src_mode=src_mode)
    564         with DatasetManager(*args, **dm_params) as DM:
--> 565             result = func(*DM.sources, **kwargs)
    566             if out is not None:
    567                 DM.out[...] = result

~/Projects/gpu-slic/survos2/improc/cuda.py in wrapper(keep_gpu, *args, **kwargs)
     37     @wraps(func)
     38     def wrapper(*args, keep_gpu=False, **kwargs):
---> 39         r = func(*args, **kwargs)
     40         return asgpuarray(r, dtype) if keep_gpu else asnparray(r, dtype)
     41     return wrapper

~/Projects/gpu-slic/survos2/improc/regions/slic.py in slic3d(data, nsp, sp_shape, compactness, sigma, spacing, max_iter, postprocess)
     29 
     30     with open(op.join(__dirname__, 'kernels', 'slic3d.cu'), 'r') as f:
---> 31         _mod_conv = SourceModule(f.read())
     32         gpu_slic_init = _mod_conv.get_function('init_clusters')
     33         gpu_slic_expectation = _mod_conv.get_function('expectation')

/scratch/ovs72384/anaconda3/envs/gpu-slic/lib/python3.7/site-packages/pycuda/compiler.py in __init__(self, source, nvcc, options, keep, no_extern_c, arch, code, cache_dir, include_dirs)
    292 
    293         from pycuda.driver import module_from_buffer
--> 294         self.module = module_from_buffer(cubin)
    295 
    296         self._bind_module()

LogicError: cuModuleLoadDataEx failed: an illegal memory access was encountered -
```

Interestingly, `sk_slic` drops out and refuses to segment the array to more than 250_000 groups. For example

In [7]:
sk_labels = sk_slic(blob, n_segments=5_000_000, compactness=1)
print(len(np.unique(sk_labels))) #250000

/scratch/ovs72384/anaconda3/envs/cuda-slic/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: skimage.measure.label's indexing starts from 0. In future version it will start from 1. To disable this warning, explicitely set the `start_label` parameter to 1.
  """Entry point for launching an IPython kernel.


360000




### 2. input array of size less than (32, 32, 32) fails with "`IndexError`: too many indices for array"

In [8]:
if RUN_TESTS:
    blob = data.binary_blobs(length=32, n_dim=3, seed=2)
    blob = np.float32(blob)
    cuda_labels = cuda_slic(blob, n_segments=2, compactness=0.5)
    print(len(np.unique(cuda_labels)))

1


```
---------------------------------------------------------------------------
IndexError                                Traceback (most recent call last)
<ipython-input-22-0316eb65c967> in <module>
      1 blob = data.binary_blobs(length=31, n_dim=3, seed=2)
      2 blob = img_as_float32(blob)
----> 3 cuda_labels = cuda_slic(blob, nsp=10, compactness=30)
      4 print(len(np.unique(cuda_labels)))

~/Projects/gpu-slic/survos2/improc/utils.py in wrapper(out, src_mode, *args, **kwargs)
    563                          fillvalue=fillvalue, src_mode=src_mode)
    564         with DatasetManager(*args, **dm_params) as DM:
--> 565             result = func(*DM.sources, **kwargs)
    566             if out is not None:
    567                 DM.out[...] = result

~/Projects/gpu-slic/survos2/improc/cuda.py in wrapper(keep_gpu, *args, **kwargs)
     37     @wraps(func)
     38     def wrapper(*args, keep_gpu=False, **kwargs):
---> 39         r = func(*args, **kwargs)
     40         return asgpuarray(r, dtype) if keep_gpu else asnparray(r, dtype)
     41     return wrapper

~/Projects/gpu-slic/survos2/improc/regions/slic.py in slic3d(data, nsp, sp_shape, compactness, sigma, spacing, max_iter, postprocess)
     85     if postprocess:
     86         min_size = int(np.prod(_sp_shape) / 10.)
---> 87         r = merge_small(asnparray(data), r, min_size)
     88         binlab = np.bincount(r.ravel())
     89 

~/Projects/gpu-slic/survos2/improc/utils.py in wrapper(out, src_mode, *args, **kwargs)
    563                          fillvalue=fillvalue, src_mode=src_mode)
    564         with DatasetManager(*args, **dm_params) as DM:
--> 565             result = func(*DM.sources, **kwargs)
    566             if out is not None:
    567                 DM.out[...] = result

~/Projects/gpu-slic/survos2/improc/utils.py in wrapper(*args, **kwargs)
    517     @wraps(func)
    518     def wrapper(*args, **kwargs):
--> 519         r = func(*args, **kwargs)
    520         return r is None or asnparray(r, dtype=dtype)
    521     return wrapper

~/Projects/gpu-slic/survos2/improc/regions/ccl.py in merge_small(data, labels, min_size, **kwargs)
     80         data = data[..., None]
     81     assert data.ndim == labels.ndim + 1
---> 82     return _merge_small3d(data, labels, labels.max()+1, min_size)
     83 
     84 

~/Projects/gpu-slic/survos2/improc/regions/_ccl.pyx in improc.superregions._ccl._merge_small3d()

IndexError: too many indices for array

```

In [9]:
## 3. lets check if the code is diterministic

In [14]:
if RUN_TESTS:
    blob = data.binary_blobs(length=10, n_dim=3, seed=2)
    blob = np.float32(blob)
    all_close = []
    for i in range(10):
        cuda_labels1 = cuda_slic(blob, n_segments=100, compactness=0.1, enforce_connectivity=False)
        cuda_labels2 = cuda_slic(blob, n_segments=100, compactness=0.1, enforce_connectivity=False)
        all_close.append((cuda_labels1 == cuda_labels2).all())
    print(all_close)

[True, False, True, True, True, True, True, True, True, True]


```
---------------------------------------------------------------------------
AssertionError                            Traceback (most recent call last)
<ipython-input-38-af438af83248> in <module>
      4 cuda_labels2 = cuda_slic(blob, nsp=32, compactness=30)
      5 
----> 6 assert np.allclose(cuda_labels1, cuda_labels2)

AssertionError: 
```

as expected. This basically confirms to me that there are memory race errors in the code.

This is likely related to Faliure mode 1. as you expect data races not to raise errors under
ordinary circomstances.

## Performance Benchmarks
Performance characteristics can reveal important bugs in the code base. So lets see how `cuda_slic` and `sk_slic` compare.

Running these benchmarks uncovered errors when the cuda_slic functin is run consecutively with many different inputs.
This indicates that the GPU memory is retaining state from previous calls that cause a subsequent kernel calls to fail.

Lets keep a record of this data:

In [15]:
import pandas as pd

In [ ]:
DO_BENCHMARK = True
SAVE_BENCHMARK = False

def generate_benchmark():
    df = pd.DataFrame(columns=["bytes", "n_segments", "cuda_time", "skimage_time"])
    lengths = [100, 200, 300, 400, 500]
    #lengths = range(100, 450,30)
    for i in range(len(lengths)):
        row = []
        row.append(lengths[i]**3*4) #number of bytes to be processed
        n_segments = lengths[i]**3/3**3
        row.append(n_segments)
        blob = data.binary_blobs(length=lengths[i], n_dim=3, seed=2)
        blob = np.float32(blob)
        print(f"cuda time for length {lengths[i]}:")
        measurement1 = %timeit -n1 -r1 -o cuda_slic(blob, n_segments=n_segments, multichannel=False, compactness=1)
        row.append(measurement1.average)
        print(f"skimage time for length {lengths[i]}:")
        measurement2 = %timeit -n1 -r1 -o sk_slic(blob, n_segments=n_segments, \
                                                  compactness=1, multichannel=False, \
                                                  max_iter=5, start_label=1)
        row.append(measurement2.average)
        df.loc[i] = row
    return df
if DO_BENCHMARK:
    df = generate_benchmark()

cuda time for length 100:
134 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
skimage time for length 100:
1.22 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
cuda time for length 200:
765 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
skimage time for length 200:
10.3 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
cuda time for length 300:
2.26 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
skimage time for length 300:
34.7 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
cuda time for length 400:
4.98 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
skimage time for length 400:
1min 21s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
cuda time for length 500:
9.42 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
skimage time for length 500:
